In [ ]:
'''
Moedl Used - Xception on Level-2 images with 36 tiles & with Less no. of Class 0 & 1 data
and rescale
'''

In [ ]:
!pip install tensorflow-addons==0.9.1

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Metric
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow.keras.applications import ResNet152, DenseNet121, InceptionResNetV2, Xception
import os
from tqdm import tqdm
import seaborn as sns

In [ ]:
# !pip install efficientnet
# import efficientnet.tfkeras as efn

In [ ]:
train_df = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv')
print(train_df.shape)
train_df.head()

In [ ]:
!ls ../input

In [ ]:
train_images_path = '../input/panda-cancer-l2-sqtile32-v1/train_squaretile_images/'

In [ ]:
img = plt.imread(train_images_path + train_df.loc[0]['image_id'] + '.png')
plt.imshow(img)
print(img.shape)

In [ ]:
train_df["image_path"] = train_df["image_id"].apply(lambda x: x + '.png')

Creating DataFrame for only available images in train directory

In [ ]:
new_img_df = pd.DataFrame()

In [ ]:
train_tile_images = list(os.listdir(train_images_path))

In [ ]:
len(train_tile_images)

In [ ]:
for i in tqdm(range(train_df.shape[0])):
    if train_df['image_path'][i] in train_tile_images:
        new_img_df = new_img_df.append(train_df.loc[i])

In [ ]:
new_img_df.info()

Code for Creating New Directories based on classes

In [ ]:
new_img_df[new_img_df['isup_grade'] == 0].sample(frac=0.451)
new_img_df[new_img_df['isup_grade'] == 1].sample(frac=0.488)
new_img_df['isup_grade'].value_counts()

In [ ]:
train_df_not_01 = new_img_df[(new_img_df['isup_grade'] != 0) & (new_img_df['isup_grade'] != 1)]
train_class0 = new_img_df[new_img_df['isup_grade'] == 0].sample(frac=0.451)
train_class1 = new_img_df[new_img_df['isup_grade'] == 1].sample(frac=0.488)

print(train_df_not_01['isup_grade'].value_counts())
print(train_class0['isup_grade'].value_counts())
print(train_class1['isup_grade'].value_counts())

In [ ]:
final_train = train_df_not_01
final_train = final_train.append(train_class0)
final_train = final_train.append(train_class1)
final_train.reset_index(inplace = True) 

In [ ]:
final_train.info()

In [ ]:
final_train['isup_grade'].value_counts()

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(final_train["image_path"], final_train["isup_grade"], test_size = 0.15, stratify = final_train["isup_grade"])

df_train = pd.DataFrame({"image_path":xtrain, "isup_grade":ytrain})
df_val = pd.DataFrame({"image_path":xval, "isup_grade":yval})

df_train["isup_grade"] = df_train["isup_grade"].astype('float')
df_val["isup_grade"] = df_val["isup_grade"].astype('float')

In [ ]:
print(df_train.shape) 
print(df_val.shape)

In [ ]:
BATCH_SIZE = 4
img_size = 768
EPOCHS = 16
nb_classes = 6

In [ ]:
LR_START = 0.00003
LR_MAX = 0.0001 * 8
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 3
LR_SUSTAIN_EPOCHS = 1
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
def get_model(model_name):
    if model_name == 'resnet_152':
        base_model =  ResNet152(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))
    elif model_name == 'dense_net_121':
        base_model =  DenseNet121(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))
    elif model_name == 'inception_resnet_v2':
        base_model =  InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))
    elif model_name == 'efficient_net_b1':
        base_model =  efn.EfficientNetB1(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))
    else:
        base_model =  Xception(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))
    x = base_model.output
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(1, activation="linear")(x) # one as it is regression model
    return Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model = get_model('xception')
model.compile(optimizer = 'adam', loss = 'mae', metrics = ['mae','mse'])

# Image Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
#        rescale = 1./255,
#        brightness_range = [1.0,1.3],
        rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
#         horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

valid_datagen = ImageDataGenerator(
#        rescale = 1./255,
#        brightness_range = [1.0,1.3],
        rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
#         horizontal_flip=True,  # randomly flip images
        vertical_flip=True)   # randomly flip images

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe = df_train,
                                               directory = train_images_path,
                                               x_col = "image_path",
                                               y_col = "isup_grade",
                                               batch_size = BATCH_SIZE,
                                               target_size =  (img_size, img_size),
                                               class_mode = 'raw')

validation_generator = valid_datagen.flow_from_dataframe(dataframe = df_val,
                                                    directory = train_images_path,
                                                    x_col = "image_path",
                                                    y_col = "isup_grade",
                                                    batch_size = BATCH_SIZE, 
                                                    target_size = (img_size, img_size),
                                                    class_mode = 'raw')

print(train_generator.__getitem__(0)[0].shape)
print(train_generator.__getitem__(0)[1].shape)

ar = train_generator.__getitem__(0)[1]
print(ar[0])
print(ar[1])
print(ar[2])
print(ar[3])
print(np.argmax(ar[0]))
print(np.argmax(ar[1]))
print(np.argmax(ar[2]))
print(np.argmax(ar[3]))

ar = train_generator.__getitem__(0)[1]
print(np.argmax(ar[0]))
print(np.argmax(ar[1]))
print(np.argmax(ar[2]))
print(np.argmax(ar[3]))

In [ ]:

# checkpoint
filepath="xception-reg-{epoch:02d}-{val_mae:.2f}.hdf5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_mae', verbose=1, save_best_only=False, mode='auto')
#callbacks_list = [checkpoint]

In [ ]:
# %%time
history = model.fit_generator(
            generator = train_generator, 
            steps_per_epoch = (df_train.shape[0] // BATCH_SIZE),
            epochs=EPOCHS,
            validation_data = validation_generator, 
            validation_steps = (df_val.shape[0] // BATCH_SIZE),
            callbacks=[lr_callback, checkpoint_callback]
)

In [ ]:
acc = history.history['mae']
val_acc = history.history['val_mae']

loss = history.history['loss']
val_loss = history.history['val_loss']

mse = history.history['mse']
val_mse = history.history['val_mse']

epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training mae')
plt.plot(epochs, val_acc, 'r', label='Validation mae')
plt.title('Training and validation MAE')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.figure()
 
plt.plot(epochs, mse, 'b', label='Training mse')
plt.plot(epochs, val_mse, 'r', label='Validation mse')
plt.title('MSE - Training and validation score')
plt.legend()

plt.show()

In [ ]:
model.save('xception_reg_level2_36_tiles_less_data.h5')